## Populating haloes with galaxies

In [1]:
# science imports
import numpy as np
import py21cmfast as p21c
from astropy import units as u
from astropy.constants import c
from astropy.cosmology import Planck18, z_at_value

In [2]:
# plotting imports
import matplotlib.pyplot as plt
rc = {"font.family" : "serif", 
    "mathtext.fontset" : "stix"}
plt.rcParams.update(rc)
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
plt.rcParams.update({'font.size': 14})
plt.style.use('dark_background')
import matplotlib as mpl
label_size = 20
font_size = 30
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size

Set initial conditions, cosmology, and astrophysical parameters.

In [3]:
# instantiate a relatively small simulation box
inputs = p21c.InputParameters.from_template('latest-dhalos', random_seed=24,).evolve_input_structs(
    SAMPLER_MIN_MASS=1e9, BOX_LEN=100, DIM=200, HII_DIM=50, USE_TS_FLUCT=False, INHOMO_RECO=False,
    HALOMASS_CORRECTION=1., AVG_BELOW_SAMPLER=True, USE_EXP_FILTER=False, USE_UPPER_STELLAR_TURNOVER=False,
    CELL_RECOMB=False, R_BUBBLE_MAX=15.).clone(
    node_redshifts=(6,7,8)
)

# create the initial conditions
init_box = p21c.compute_initial_conditions(
    inputs=inputs,
)

Generate lists of halos at our node redshifts: $6$, $7$, $8$

In [4]:
# now we scroll through each redshift in descending order and generate halo lists
halo_lists = []
descendant_halos = None
for z in inputs.node_redshifts[::-1]:
    # generate the halo field
    halo_list = p21c.determine_halo_list(
        redshift=z,
        initial_conditions=init_box,
        inputs=inputs,
        descendant_halos=descendant_halos
    )
    descendant_halos = halo_list
    halo_lists.append(halo_list)

Each halo list contains the halos *coordinates*, *masses*, *stellar masses*, and *star formation rates*.

The latter two properties are encoded in random numbers `star_rng` and `sfr_rng`, which we must pass through a decoder function in order to compute the corresponding physical quantities. Each of these numbers quantifies how far removed a halo's property is from the mean. 

Many studies of high-redshift galaxies do not account for stochasticity in the stellar mass-to-halo mass relation, which can lead to significant problems (see Nikolic et al. 2024 https://arxiv.org/abs/2406.15237).

To better understand this, let us investigate the impact of stochasticity on the UV luminosity function by computing this quantity first with a fixed relation, and then using the scatter considered in `21cmFAST`.

In [5]:
def get_stellar_mass(halo_masses, stellar_rng):
    sigma_star =  inputs.astro_params.SIGMA_STAR
    mp1 = 1e10
    mp2 = 10**(inputs.astro_params.UPPER_STELLAR_TURNOVER_MASS)
    m_turn = 10**(inputs.astro_params.M_TURN)
    a_star = inputs.astro_params.ALPHA_STAR
    a_star2 = inputs.astro_params.UPPER_STELLAR_TURNOVER_INDEX
    f_star10 = 10**inputs.astro_params.F_STAR10
    omega_b = inputs.cosmo_params.OMb
    omega_m = inputs.cosmo_params.OMm
    baryon_frac = omega_b/omega_m
    
    high_mass_turnover = ((mp2/mp1)**a_star + (mp2/mp1)**a_star2)/((halo_masses/mp2)**(-1*a_star)+(halo_masses/mp2)**(-1*a_star2))
    stoc_adjustment_term = 0.5*sigma_star**2
    low_mass_turnover = np.exp(-1*m_turn/halo_masses + stellar_rng*sigma_star - stoc_adjustment_term)
    stellar_mass = f_star10 * baryon_frac * halo_masses * (high_mass_turnover * low_mass_turnover)
    return stellar_mass

def get_sfr(stellar_mass, sfr_rng, z):
    sigma_sfr_lim = inputs.astro_params.SIGMA_SFR_LIM
    sigma_sfr_idx = inputs.astro_params.SIGMA_SFR_INDEX
    t_h = 1/Planck18.H(z).to('s**-1').value
    t_star = inputs.astro_params.t_STAR
    sfr_mean = stellar_mass / (t_star * t_h)
    sigma_sfr = sigma_sfr_idx * np.log10(stellar_mass/1e10) + sigma_sfr_lim
    sigma_sfr[sigma_sfr < sigma_sfr_lim] = sigma_sfr_lim
    stoc_adjustment_term = sigma_sfr * sigma_sfr / 2. # adjustment to the mean for lognormal scatter
    sfr_sample = sfr_mean * np.exp(sfr_rng*sigma_sfr - stoc_adjustment_term)
    return sfr_sample

In [6]:
halo_masses_list        = []
stellar_masses_stoch    = []
sfr_stoch               = []
stellar_masses_no_stoch = []
sfr_no_stoch            = []

for halo_list in halo_lists:
    # get the halo masses
    halo_masses = halo_list.get('halo_masses')
    # get the stellar masses and star formation rates
    star_rng = halo_list.get('star_rng')
    sfrs = halo_list.get('sfr_rng')

    # purge zero values
    star_rng = star_rng[halo_masses > 0]
    sfrs = sfrs[halo_masses > 0]
    halo_masses = halo_masses[halo_masses > 0]

    sm_stoch = get_stellar_mass(halo_masses, star_rng)
    sf_stoch = get_stellar_mass(halo_masses, sfrs)
    sm_no_stoch = get_stellar_mass(halo_masses, np.zeros_like(halo_masses))
    sf_no_stoch = get_stellar_mass(halo_masses, np.zeros_like(halo_masses))
    
    # append to the lists
    stellar_masses_stoch    += [sm_stoch]
    sfr_stoch                += [sf_stoch]
    stellar_masses_no_stoch += [sm_no_stoch]
    sfr_no_stoch             += [sf_no_stoch]
    halo_masses_list        += [halo_masses]

In [ ]:
# visualize stochasticity in each relationship
fig, axs = plt.subplots(2, 3, figsize=(12, 8), sharex=True, sharey=True)

axs[0,0].plot(np.log10(halo_masses_list[0]), np.log10(stellar_masses_stoch[0]), 'o', color='cyan', markersize=1, alpha=0.5, label='Stochastic')
axs[0,0].plot(np.log10(halo_masses_list[0]), np.log10(stellar_masses_no_stoch[0]), 'o', color='red', markersize=1, alpha=0.5, label='Stochastic')